In [ ]:
#!source ~/.bashrc

In [34]:
# openai 테스트
from openai import OpenAI
import os
#OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')
print(OPENAI_API_KEY)
#client = OpenAI(api_key="")
#client = OpenAI(api_key=OPENAI_API_KEY)

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message)

None
ChatCompletionMessage(content="In the realm of codes and syntax, bare,\nThere lies a concept, beyond compare.\nRecursion it's called, a loop divine,\nA pattern of function, cleverly designed.\n\nLike a mirror reflecting itself anew,\nRecursion calls the function, not a few.\nIt's a tale of a function that calls its own name,\nA dance of repetition, a never-ending game.\n\nThe base case marks the end, a boundary found,\nBreaking the cycle, on solid ground.\nRecursive calls, each a step smaller,\nSolving problems complex, like a skilled brawler.\n\nThrough layers of calls, the solution unfolds,\nA tower of frames, a story untold.\nInfinite possibilities, yet contained in recursion,\nA loop of logic, a programmer's vision.\n\nSo heed the call of recursion's embrace,\nIn the realm of programming, finding grace.\nA concept profound, in its elegant chore,\nRecursion, the muse, forevermore.", role='assistant', function_call=None, tool_calls=None)


In [10]:
!pip install llmlingua

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://repo.kakao.com/pypi/simple/, http://kakaobrain-pypi.dev.9rum.cc/, https://pypi.org/simple


# GPT-3.5-Turbo on GSM8K


In [14]:
# 필요한 package import
import openai
import re
import time

import numpy as np

from tqdm import tqdm
from datasets import load_dataset

In [18]:
# data loader
!git clone https://github.com/FranxYao/chain-of-thought-hub.git
gsm8k = load_dataset('gsm8k', 'main')
validation_index = np.load('chain-of-thought-hub/gsm8k/lib_prompt/validation_index.npy')
validation_data = gsm8k['train'].select(validation_index)
gsm8k_test = gsm8k['test']

fatal: destination path 'chain-of-thought-hub' already exists and is not an empty directory.


In [19]:
# 데이터 확인
gsm8k['train'][0]['question']


'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?'

In [21]:
# 테스트 데이터
gsm8k_test = gsm8k['test']

In [23]:
prompt_complex = open('chain-of-thought-hub/gsm8k/lib_prompt/prompt_hardest.txt').read()
print(prompt_complex)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [67]:
from tenacity import (
    retry,
    stop_after_attempt,
    wait_chain,
    wait_fixed
) 

@retry(wait=wait_chain(*[wait_fixed(3) for i in range(3)] +
                       [wait_fixed(5) for i in range(2)] +
                       [wait_fixed(10)]))
def completion_with_backoff(**kwargs):
    # 난 왜 이렇게 호출해야 통신이 될까??
    return client.chat.completion.create(**kwargs)
    return openai.ChatCompletion.create(**kwargs)

In [59]:
def test_answer(pred_str, ans_str):
    pattern = '\d*\.?\d+'
    pred = re.findall(pattern, pred_str)
    if(len(pred) >= 1):
        # print(pred_str)
        pred = pred[-1]
        gold = re.findall(pattern, ans_str)
        # print(ans_str)
        gold = gold[-1]
        return pred == gold
    else: return False

def parse_pred_ans(filename):
    with open(filename) as fd: lines = fd.readlines()
    am, a = None, None
    num_q, acc = 0, 0
    current_mode = 'none'
    questions = []
    ans_pred = []
    ans_gold = []
    for l in lines:
        if(l.startswith('Q: ')):
            if(am is not None and a is not None):
                questions.append(q)
                ans_pred.append(am)
                ans_gold.append(a)
                if(test_answer(am, a)):
                    acc += 1
            current_mode = 'q'
            q = l
            num_q += 1
        elif(l.startswith('A_model:')):
            current_mode = 'am'
            am = l
        elif(l.startswith('A:')):
            current_mode = 'a'
            a = l
        else:
            if(current_mode == 'q'): q += l
            elif(current_mode == 'am'): am += l
            elif(current_mode == 'a'): a += l
            else:
                raise ValueError(current_mode)
                
    questions.append(q)
    ans_pred.append(am)
    ans_gold.append(a)
    if(test_answer(am, a)):
        acc += 1
    print('num_q %d correct %d ratio %.4f' % (num_q, acc, float(acc / num_q)))
    return questions, ans_pred, ans_gold

def test_finished(ans_model):
    if('answer is' in ans_model): return True
    else: return False

def extract_ans(ans_model):
    ans_model = ans_model.split('\n')
    ans = []
    residual = []
    for li, al in enumerate(ans_model):
        ans.append(al)
        if('answer is' in al):
            break
    residual = list(ans_model[li + 1:])
    ans = '\n'.join(ans)
    residual = '\n'.join(residual)
    return ans, residual

In [60]:
prompt_q = prompt_complex + '\nQuestion: ' + gsm8k_test[1]['question'] + '\n'
print(prompt_q)

Question: Angelo and Melanie want to plan how many hours over the next week they should study together for their test next week. They have 2 chapters of their textbook to study and 4 worksheets to memorize. They figure out that they should dedicate 3 hours to each chapter of their textbook and 1.5 hours for each worksheet. If they plan to study no more than 4 hours each day, how many days should they plan to study total over the next week if they take a 10-minute break every hour, include 3 10-minute snack breaks each day, and 30 minutes for lunch each day?
Let's think step by step
Angelo and Melanie think they should dedicate 3 hours to each of the 2 chapters, 3 hours x 2 chapters = 6 hours total.
For the worksheets they plan to dedicate 1.5 hours for each worksheet, 1.5 hours x 4 worksheets = 6 hours total.
Angelo and Melanie need to start with planning 12 hours to study, at 4 hours a day, 12 / 4 = 3 days.
However, they need to include time for breaks and lunch. Every hour they want 

In [66]:
response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
        {"role": "system", "content": "Follow the given examples and answer the question."},
        {"role": "user", "content": prompt_q},
    ]
)

In [68]:
response.choices[0].message.content

"Let's think step by step\nIf Janet's ducks lay 16 eggs per day, she uses 3 for breakfast and 4 for muffins, totaling 3+4 = 7 eggs.\nThis means the remainder of eggs for sale is 16-7 = 9 eggs\nAt the farmers' market, each fresh duck egg is sold for $2, which means Janet makes 9 * $2 = $18 daily at the farmers' market.\nTherefore, she makes $18 in dollars every day at the farmers' market."

In [69]:
print(response.choices[0].message.content)

Let's think step by step
If Janet's ducks lay 16 eggs per day, she uses 3 for breakfast and 4 for muffins, totaling 3+4 = 7 eggs.
This means the remainder of eggs for sale is 16-7 = 9 eggs
At the farmers' market, each fresh duck egg is sold for $2, which means Janet makes 9 * $2 = $18 daily at the farmers' market.
Therefore, she makes $18 in dollars every day at the farmers' market.


In [79]:
i = 0
with open('outputs/test_gpt_3.5_turbo_complex.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_complex + '\nQuestion: ' + q + '\n'  
        
        #response = completion_with_backoff(
        #      model="gpt-3.5-turbo",
        #      messages=[
        #            {"role": "system", "content": "Follow the given examples and answer the question."},
        #            {"role": "user", "content": prompt_q},
        #        ]
        #    )
        response = client.chat.completions.create(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": "Follow the given examples and answer the question."},
                    {"role": "user", "content": prompt_q},
                ]
            )
        ans_model = response.choices[0].message.content
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        # if(i == 2): break

 85%|████████▌ | 1122/1319 [54:11<09:30,  2.90s/it] 


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_397/406578191.py", line 15, in <module>
    response = client.chat.completions.create(
  File "/home/bc-user/.local/lib/python3.10/site-packages/openai/_utils/_utils.py", line 301, in wrapper
  File "/home/bc-user/.local/lib/python3.10/site-packages/openai/resources/chat/completions.py", line 598, in create
    user: str | NotGiven = NOT_GIVEN,
  File "/home/bc-user/.local/lib/python3.10/site-packages/openai/_base_client.py", line 1096, in post
    stream=stream,
  File "/home/bc-user/.local/lib/python3.10/site-packages/openai/_base_client.py", line 856, in request
    ) -> None:
  File "/home/bc-user/.local/lib/python3.10/site-packages/openai/_base_client.py", line 908, in _request
    stream_cls: Type[_StreamT] | None = None,
openai.AuthenticationError: Error code: 401 

In [ ]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_complex.txt')

i = 0
with open('outputs/test_gpt_3.5_turbo_complex_temp_0.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_complex + '\nQuestion: ' + q + '\n'  
        
        response = client.chat.completions.create(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": "Follow the given examples and answer the question."},
                    {"role": "user", "content": prompt_q},
                ],
                temperature=0
            )
        ans_model = response.choices[0].message.content
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        # if(i == 2): break

_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_complex_temp_0.txt')


In [ ]:
prompt_original = open('../gsm8k/lib_prompt/prompt_original.txt').read()

i = 0
with open('outputs/test_gpt_3.5_turbo_original_temp_0.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_original + '\nQuestion: ' + q + '\n'  
        
        response = client.chat.completions.create(
              model="gpt-3.5-turbo",
              messages=[
                    {"role": "system", "content": "Follow the given examples and answer the question."},
                    {"role": "user", "content": prompt_q},
                ],
                temperature=0
            )
        ans_model = response.choices[0].message.content
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        # if(i == 2): break

_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_original_temp_0.txt')


In [ ]:
def make_dialog_prompt(prompt):
    messages = []
    messages.append({"role": "system", "content": "Follow the given examples and answer the question."})
    cases = prompt.split("\n\n")
    for c in cases[:-1]:
        question = c.split("\n")[:2]
        messages.append({"role": "user", "content": "\n".join(question)})
        answer = c.split("\n")[2:]
        messages.append({"role": "assistant", "content": "\n".join(answer)})
    messages.append({"role": "user", "content": cases[-1] + "Let's think step by step"})
    return messages

In [ ]:
i = 0
with open('outputs/test_gpt_3.5_turbo_original_dialog_icl.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_original + '\nQuestion: ' + q + '\n'
        dialog_prompt = make_dialog_prompt(prompt_q)
        
        response = client.chat.completions.create(
              model="gpt-3.5-turbo",
              messages=dialog_prompt,
              temperature=0
            )
        ans_model = response.choices[0].message.content
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        # if(i == 2): break

In [ ]:
_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_original_dialog_icl.txt')


In [ ]:
i = 0
with open('outputs/test_gpt_3.5_turbo_complex_dialog_icl.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_complex + '\nQuestion: ' + q + '\n'
        dialog_prompt = make_dialog_prompt(prompt_q)
        
        response = client.chat.completions.create(
              model="gpt-3.5-turbo",
              messages=dialog_prompt,
              temperature=0
            )
        ans_model = response.choices[0].message.content
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        # if(i == 2): break

_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_complex_dialog_icl.txt')

In [ ]:
# LLMLingual-2를 이용해서 compression 수행해보기
# 300 token으로 줄이기 -> iterative_size로 컨트롤 되는 듯 https://github.com/microsoft/LLMLingua/blob/441120178e2ae51e15db4a63355b6f713e1f2fc1/llmlingua/prompt_compressor.py#L486

from llmlingua import PromptCompressor

## Or use LLMLingua-2-small model
llm_lingua = PromptCompressor(
    model_name="microsoft/llmlingua-2-bert-base-multilingual-cased-meetingbank",
    use_llmlingua2=True, # Whether to use llmlingua-2
)

with open('outputs/test_gpt_3.5_turbo_complex_dialog_complession_300_token.txt', 'w') as fd:
    for q, a in tqdm(zip(gsm8k_test['question'], gsm8k_test['answer']), 
                               total=len(gsm8k_test['question'])):
        
        prompt_q = prompt_complex + '\nQuestion: ' + q + '\n'
        compressed_prompt = llm_lingua.compress_prompt(prompt_q, rate=0.5, iterative_size=300, force_tokens = ['\n', '?'])

        print(f"prompt_q is {prompt_q} \n\ncompress_prompt is {compressed_prompt}")

        dialog_prompt = make_dialog_prompt(compressed_prompt)
        
        response = client.chat.completions.create(
              model="gpt-3.5-turbo",
              messages=dialog_prompt,
              temperature=0
            )
        ans_model = response.choices[0].message.content
        ans_, residual = extract_ans(ans_model)
            
        fd.write('Q: %s\nA_model:\n%s\nA:\n%s\n\n' % (q, ans_, a))
        i += 1
        # if(i == 2): break

_, _, _ = parse_pred_ans('outputs/test_gpt_3.5_turbo_complex_dialog_complession_300_token.txt')
